In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',None)

import seaborn as sns
import matplotlib.pyplot as plt

In [23]:
df = pd.read_excel('../data/ViratKohli-ODI.xlsx') ## This is the Main Data. 

In [7]:
df.tail()

,Match,Innings,Date,M/Inns,Posn,Versus,Ground,How Dismissed,Runs,B/F,S/R,Progressive-Runs,Progressive-B/F,Progressive-Avg,Progressive-S/R
304,305,293.0,2025-10-25,2nd,3,Australia,Sydney Cricket Ground,not out,74*,81,91.36,14255,15285,57.71,93.26
305,306,294.0,2025-11-30,1st,3,South Africa,JSCA International Stadium Complex,c R D Rickelton b N Burger,135,120,112.5,14390,15405,58.02,93.41
306,307,295.0,2025-12-03,1st,3,South Africa,Shaheed Veer Narayan Sing International Stadium,c A K Markram b L Ngidi,102,93,109.68,14492,15498,58.20,93.51
307,308,296.0,2025-12-06,2nd,3,South Africa,Dr YS Rajasekhara Reddy Cricket Stadium,not out,65*,45,144.44,14557,15543,58.46,93.66
308,309,297.0,2026-01-11,2nd,3,New Zealand,Kotambi Stadium,c M G Bracewell b K A Jamieson,93,91,102.2,14650,15634,58.60,93.71


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Match             309 non-null    object        
 1   Innings           297 non-null    float64       
 2   Date              309 non-null    datetime64[ns]
 3   M/Inns            305 non-null    object        
 4   Posn              309 non-null    object        
 5   Versus            309 non-null    object        
 6   Ground            309 non-null    object        
 7   How Dismissed     309 non-null    object        
 8   Runs              309 non-null    object        
 9   B/F               309 non-null    object        
 10  S/R               309 non-null    object        
 11  Progressive-Runs  309 non-null    int64         
 12  Progressive-B/F   309 non-null    int64         
 13  Progressive-Avg   309 non-null    float64       
 14  Progressive-S/R   309 non-

In [9]:
df.describe().transpose()

,count,mean,min,25%,50%,75%,max,std
Innings,297.0,149.0,1.0,75.0,149.0,223.0,297.0,85.880731
Date,309,2016-03-27 22:12:48.932038912,2008-08-18 00:00:00,2012-02-14 00:00:00,2015-03-10 00:00:00,2019-06-27 00:00:00,2026-01-11 00:00:00,NaN
Progressive-Runs,309.0,6935.527508,12.0,3001.0,6495.0,11159.0,14650.0,4390.918282
Progressive-B/F,309.0,7604.187702,22.0,3588.0,7216.0,11985.0,15634.0,4611.716036
Progressive-Avg,309.0,52.09932,12.0,49.17,52.17,57.87,60.31,6.767183
Progressive-S/R,309.0,87.967023,54.55,83.94,89.91,92.93,93.85,6.426389


### cleaning "Runs"

In [10]:
runs = []
for i in df['Runs']:
    if str(i).rfind('*')!= -1:
        runs.append(i.split('*')[0])
    elif str(i).rfind('-')!= -1:
        runs.append(0)
    elif str(i).rfind('*') == -1:
        runs.append(i)
df['Runs'] = runs
df['Runs'] = df['Runs'].astype('int64')

### Adding "Captaincy"

In [11]:
captain = []
for i in df['Match']:
    if i.rfind('(c)')!=-1:
        captain.append('Yes')
    elif i.rfind('(c)')==-1:
        captain.append('No')
df['Captain'] = captain

### Cleaning "B/F"

In [12]:
df['B/F'].dtype

dtype('O')

In [13]:
bf = []
for i in df['B/F']:
    if str(i) == '-':
        bf.append(0)
    else:
        bf.append(i)
df['B/F'] = bf
df['B/F'] = df['B/F'].astype('int64')

### Cleaning "S/R"

In [14]:
df['S/R'].dtype

dtype('O')

In [15]:
sr = []
for i in df['S/R']:
    if str(i) == '-':
        sr.append(0)
    else:
        sr.append(i)
df['S/R'] = sr
df['S/R'] = df['S/R'].astype('float64')

### Creating "Dismissal Type"

In [16]:
dt = []
for i in df['How Dismissed']:
    if (i == 'run out'):
        dt.append(i) 
    elif str(i).rfind('hit wicket') != -1:
        dt.append('hit wicket')
    elif (i != 'did not bat') and (i!='not out'):
        dt.append(i.split(' ')[0])
    else:
        dt.append(i)
        
df['Dismissal Type'] = dt
df['Dismissal Type'] = df['Dismissal Type'].map({'lbw':'LBW' , 'c':'Caught',
                                                'run out':'Run Out' , 'b':'Bowled',
                                                'not out':'Not Out', 'did not bat':'Did Not Bat',
                                                'st':'Stumped', 'hit wicket':'Hit Wicket'})

### Cleaning "Innings"

In [17]:
df['M/Inns'].fillna('N/A - No Result',inplace=True,axis=0)

C:\Users\Sharan Kalyan\AppData\Local\Temp\ipykernel_14404\1393352032.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['M/Inns'].fillna('N/A - No Result',inplace=True,axis=0)


### Cleaning "Posn"

In [18]:
df.Posn.unique()

array([2, 1, 7, 4, '-', 3, 5, 6], dtype=object)

In [19]:
df['Posn'] = df['Posn'].replace({'-':0})

C:\Users\Sharan Kalyan\AppData\Local\Temp\ipykernel_14404\144243979.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Posn'] = df['Posn'].replace({'-':0})


### Nulls

In [20]:
df.isnull().sum()

Match                0
Innings             12
Date                 0
M/Inns               0
Posn                 0
Versus               0
Ground               0
How Dismissed        0
Runs                 0
B/F                  0
S/R                  0
Progressive-Runs     0
Progressive-B/F      0
Progressive-Avg      0
Progressive-S/R      0
Captain              0
Dismissal Type       0
dtype: int64

### Nulls in innings indicate that we have a result for the match but Virat Kohli did not bat. 

In [21]:
df.head()

,Match,Innings,Date,M/Inns,Posn,Versus,Ground,How Dismissed,Runs,B/F,S/R,Progressive-Runs,Progressive-B/F,Progressive-Avg,Progressive-S/R,Captain,Dismissal Type
0,1,1.0,2008-08-18,1st,2,Sri Lanka,Rangiri Dambulla International Stadium,lbw b K M D N Kulasekara,12,22,54.55,12,22,12.00,54.55,No,LBW
1,2,2.0,2008-08-20,2nd,2,Sri Lanka,Rangiri Dambulla International Stadium,c C K Kapugedera b T Thushara,37,67,55.22,49,89,24.50,55.06,No,Caught
2,3,3.0,2008-08-24,1st,1,Sri Lanka,R Premadasa Stadium,run out,25,38,65.79,74,127,24.67,58.27,No,Run Out
3,4,4.0,2008-08-27,1st,1,Sri Lanka,R Premadasa Stadium,b T Thushara,54,66,81.82,128,193,32.00,66.32,No,Bowled
4,5,5.0,2008-08-29,2nd,1,Sri Lanka,R Premadasa Stadium,lbw b K M D N Kulasekara,31,46,67.39,159,239,31.80,66.53,No,LBW


### Save Clean data

### This data is fed to Tableau Dashboard Along with 

In [22]:
df.to_excel('../Data/ODI-Clean.xlsx',index=False)